In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns
import plotly.graph_objects as go

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Basic Visual Examination

1. Distribution of homicides by gender,
1. Distribution of homicides by age,
1. Weapons used in homicides,
1. Distribution of homicides by States and Cities,
1. Relationship between Perpetrator and Victim,
1. Relationships of homicides by races,
1. Distribution of homicides by years,
1. Women homicides by years.

In [ ]:
data = pd.read_csv("/kaggle/input/homicide-reports/database.csv")
df = data.copy()
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe().T

### The meaning of the "incident" column

In [ ]:
# df[df["Incident"]>900].sort_values(by="Incident")
# incident sütununda 900lü sayılar neden var bunu anlamaya çalışıyorum.

In [ ]:
df[(df["Agency Name"]=="Los Angeles") & (df["Agency Type"]=="Sheriff") & (df["Year"]==1980) & (df["Month"]=="August")].sort_values(by="Incident")

anlaşılan o ki; her şehirde, agency name'e ve agency type'a göre, her ay 0'dan başlanılarak, gerçekleşen her olay için "incident" numarası verilmiş. ama mükerrer kayıtlar var görünüyor.

Yukarıdaki tabloda "los angeles" teşkilatı, "sheriff" biriminin 1980 ağustos ayındaki cinayet dosyaları görünüyor. Toplam 20 tane.

Fakat bazı yerlerde 900 küsürlerden başlayarak numara verilmiş. (Aşağıdaki iki tabloda görünüyor). Ancak biz verilen numaralarla ingilenmeyip, kaç satır varsa o kadar cinayet dosyası var diyerek işleme devam edeceğiz.

In [ ]:
df[(df["Agency Name"]=="Miami") & (df["Agency Type"]=="Municipal Police") & (df["Year"]==2006) & (df["Month"]=="December")].sort_values(by="Incident")

In [ ]:
#df[(df["Agency Name"]=="Orange County") & (df["Year"]==2013) & (df["Month"]=="April")].sort_values(by="Incident")

In [ ]:
df["Incident"] = 1

## 1 - Distribution of homicides by gender

In [ ]:
df["Victim Sex"].value_counts()
#df["Victim Sex"].value_counts().plot.bar();

In [ ]:
df["Perpetrator Sex"].value_counts()
#df["Perpetrator Sex"].value_counts().plot.bar();

In [ ]:
def bar_plot(variable):
    
    var = df[variable]
    
    varValue = var.value_counts()
        
    plt.figure(figsize = (9,3))
    plt.bar(varValue.index, varValue)
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.show
    #print("{}: \n {}".format(variable, varValue))
    
category1=["Victim Sex","Perpetrator Sex"]
list(map(lambda x:bar_plot(x), category1))
plt.show()

# Kategorik değişkenler için

Erkeklerin öldürme sayısı da öldürülme sayısı da yüksek iken, kadınların öldürme sayısı öldürülme sayısına göre çok düşük.

In [ ]:
df_known = df[(df['Victim Sex'] != 'Unknown') & (df['Perpetrator Sex'] != 'Unknown')]
df_gender = df_known.groupby(['Perpetrator Sex','Victim Sex']).size()
df_gender_df = df_gender.to_frame(name='Count')
df_gender_df.reset_index(inplace=True)
df_gender_df

In [ ]:
plt.figure(figsize = (9,3))
clrs = ['#EE99AC',"#6890F0"]
sns.barplot(x='Perpetrator Sex', y="Count", hue='Victim Sex', palette=clrs, data=df_gender_df);
plt.title("A Comparison of Men and Women Who are Perpetrator and Victim");


# plt.figure(figsize = (9,3))
# sns.countplot(x='Perpetrator Sex', data=df, hue='Victim Sex');
# plt.title("A Comparison of Men and Women Who are Perpetrator and Victim");

Boyut eklemeden önce, erkeklerin 400 bin cinayet işlediğini görüyoruz. Boyut ekledikten sonra kurbanların 300 bininin erkek 100 bininin ise kadın olduğunu görüyoruz.

## 2 - Distribution of homicides by age

In [ ]:
def hist_plot(variable):
    
    plt.figure(figsize=(9,3))
    plt.hist(df[variable],bins = 50)
    
    plt.xlabel(variable)
    plt.ylabel("Frequency")
    plt.title("{} Distribution with histogram". format(variable))
    plt.show
numericalVariables=["Victim Age", "Victim Count"]
list(map(lambda x: hist_plot(x), numericalVariables))
plt.show()

#Sayısal değişkenler için.

Bazı yaşların yanlış girildiğini görüyoruz. "900" küsürlü yaşlar var. Yaş aralığını görmek için daha özelleştirilmiş bir tabloya ihtiyaç var.

Victim Count sütununun ise neredeyse tamamının "0" olarak girildiğini görüyoruz.

In [ ]:
df["Incident"] = 1
age = pd.cut(df["Victim Age"],[0,18,36,54,90])

df_age = df.pivot_table(["Incident"], index =["Victim Sex",age], aggfunc=sum)
df_age

In [ ]:
df_age_f = pd.DataFrame({"0-18":df_age.iloc[0],
                         "18-36":df_age.iloc[1],
                        "36-54":df_age.iloc[2],
                        "54-90":df_age.iloc[3]})

                                    
df_age_f = df_age_f.T
df_age_f.columns = ["Incident"]

plt.figure(figsize = (9,6))
plt.subplot(2,2,1)
sns.barplot(x=df_age_f.index, y="Incident",data=df_age_f);
plt.title("Ages of Female Victims");
plt.xlabel("Age Groups");

##
df_age_m = pd.DataFrame({"0-18":df_age.iloc[4],
                         "18-36":df_age.iloc[5],
                        "36-54":df_age.iloc[6],
                        "54-90":df_age.iloc[7]})
df_age_m = df_age_m.T
df_age_m.columns = ["Incident"]

plt.figure(figsize = (9,6))
plt.subplot(2,2,2)
sns.barplot(x=df_age_m.index, y="Incident",data=df_age_m);
plt.title("Ages of Male Victims");
plt.xlabel("Age Groups");

In [ ]:
df_age_b = pd.concat([df_age_m,df_age_f],axis=1)
df_age_b.columns = ["Male", "Female"]
df_age_b.plot.bar();
plt.title("Ages of Victims");

## 3 - Weapons used in homicides

In [ ]:
df["Weapon"].value_counts()

In [ ]:
df_we = df.pivot_table(index="Year", columns="Weapon", aggfunc=sum)["Incident"]
df_we

In [ ]:
plt.figure(figsize=(9,3))
sns.barplot(data=df_we);
plt.xticks(rotation=90);

# df["Weapon"].value_counts().plot.pie();

In [ ]:
labels = df.groupby('Weapon').sum().index

explode = [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]
sizes = df.groupby('Weapon').sum()['Incident']

plt.figure(figsize = (7,7))
plt.pie(sizes, explode=explode, labels=labels, labeldistance=1.5, autopct='%1.1f%%')
plt.title('Weapons',fontsize = 15);

Pasta ne kadar büyürse büyüsün veya yazılar istediği kadar küçülsün veya uzaklaşsın istediğim görüntüyü yakalayamıyorum, bu yüzden belli bir değerden küçük olanları diğerleri olarak sınıflayacağım.

In [ ]:
w = df.groupby('Weapon').sum()['Incident']
w = pd.DataFrame(w)
#w
weapons = list(df["Weapon"].unique())

others = []
for i in weapons:
    if w.loc[i].values < 7000:
        others.append(w.loc[i].values)    
    
w.loc["Others"] = sum(others)
w_elim = w[w.Incident > 7000] 
#w_elim #eliminated

labels = w_elim.index
explode = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
sizes = w_elim['Incident']

plt.figure(figsize = (7,7))
plt.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%')
plt.title('Weapons',fontsize = 15);

## 4 - Distribution of homicides by States and Cities

In [ ]:
plt.figure(figsize = (18,8))
# df["State"].value_counts().plot.bar();

states_vis = sns.countplot(x='State', order=df['State'].value_counts().index, data=df);

for item in states_vis.get_xticklabels():
    item.set_rotation(90)

In [ ]:
city_usa = pd.read_csv("/kaggle/input/usa-cities/usa.txt")
#city_usa.head()

new = []
for i in list(city_usa.name):
    new.append(i.strip())
    
city_usa["name"] = new
city_usa.head()

In [ ]:
cities = df.groupby("City").size()
cities_df = cities.to_frame(name='count')
cities_df.reset_index(inplace=True)
cities_df.columns=(["name","count"])
cities_df.head()

# cities = df.groupby("City")["Incident"].sum()
# cities_df = pd.DataFrame(cities)
# cities_df

In [ ]:
df_map = pd.merge(cities_df, city_usa, on = "name", how='inner')
df_map.head()

In [ ]:
df_map['text'] = df_map['name'] + ' - Number of Incident: ' + (df_map['count'].astype(str)) 
limits = [0,100,500,1000,50000]
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
cities = []

fig = go.Figure()
a= -4
for i in range(len(limits)):

    df_sub = df_map[(df_map["count"]>limits[i]) & (df_map["count"]<limits[a])]
     
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_map["count"],
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(limits[i],limits[a])))
    a +=1
fig.update_layout(
        title_text = 'Homicide Numbers of USA Cities',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()

## 5 - Relationships between Perpetrator and Victim

In [ ]:
df["Relationship"].value_counts()

In [ ]:
a = df["Relationship"].replace('Unknown', np.nan)
a.value_counts().plot.barh();

# sns.countplot(x='Relationship', data=df.replace('Unknown', np.nan), order=df["Relationship"].value_counts().index)

#df["Relationship"].value_counts().plot.barh();
# bu şekilde yapınca unknown lar çok olduğu için anlamsız görünüyor.

## 6 - Relationships of homicides by races

In [ ]:
df["Perpetrator Race"].value_counts().plot.bar();

In [ ]:
df_race_pv = df.pivot_table(index=["Perpetrator Race","Victim Race"], aggfunc=sum)["Incident"]
df_race_pv = pd.DataFrame(df_race_pv)
df_race_pv.head()

In [ ]:
df_race = pd.DataFrame({"White-White":df_race_pv.loc["White","White"].values,
                 "White-Black":df_race_pv.loc["White","Black"].values,
                 "Black-White":df_race_pv.loc["Black","White"].values,
                 "Black-Black":df_race_pv.loc["Black","Black"].values})

df_race = df_race.T
df_race.columns = ["Incident"]

sns.barplot(x=df_race.index, y="Incident", data=df_race);
plt.title("Relationship Between Black and White Races");
plt.xlabel("Perpetrator Race - Victim Race");

## 7 - Distribution of homicides by years

In [ ]:
df.groupby("Year")["Month"].value_counts()
# her ay kaç cinayet işlenmiş onu buluyoruz.

# df.groupby("Year")["Month"].value_counts().sum() 
# sağlamasını yapmış oldu. toplam satır sayısına eşit oluyor.

In [ ]:
df_pv = df.pivot_table(index=["Year"], columns=["Month"], aggfunc=(lambda x : x.count()), margins = True)["Incident"]
# sütunları alfabetik sıraya göre sıralıyordu onu değiştirdik.
df_pv = df_pv.reindex(columns=["January","February","March","April","May","June","July","August","September","October","November","December","All"])
df_pv

In [ ]:
df_pv = df.pivot_table(index=["Year"], columns=["Month"],aggfunc=(lambda x : x.count()))["Incident"]
df_pv = df_pv.reindex(columns=["January","February","March","April","May","June","July","August","September","October","November","December"])

df_pv.plot();
#zaman serisi grafiği çizme

In [ ]:
sns.heatmap(df_pv);

In [ ]:
df_year = df.groupby("Year")["Incident"].sum()
df_year = pd.DataFrame(df_year)

plt.title("Number of Homicide by Years");
sns.lineplot(x=df_year.index, y= "Incident", data=df_year);

In [ ]:
df_s_y = df.pivot_table(index=["Victim Sex","Year"], aggfunc=sum)["Incident"]
df_s_y = pd.DataFrame(df_s_y)
df_s_y.head()
# df_s_y.loc["Female",1980].values
# df_s_y.loc["Female",1980].index

In [ ]:
def gender_plot(sex):
    data = df_s_y.loc[sex]

    plt.plot(data.index, data.values)
    
# gender_plot("Female")
genders = ["Male","Female"]

plt.figure(figsize = (18, 8))
for i in genders:
    gender_plot(i)

plt.title("Homicide Victims by Years and Genders");
plt.legend(genders);

## 8 - Women homicides by years

In [ ]:
df_s_r_y = df.pivot_table(index=["Victim Sex","Relationship","Year"], aggfunc=sum)["Incident"]
df_s_r_y = pd.DataFrame(df_s_r_y)
df_s_r_y.head()

In [ ]:
plt.figure(figsize = (18, 8))

def sex_rel(sex, relation):
    data = df_s_r_y.loc[sex, relation]

    plt.plot(data.index, data.values)
    
relations = ["Wife","Ex-Wife","Girlfriend","Mother","Sister","Daughter"]
             
for relation in relations:
    sex_rel("Female", relation)

plt.title("Number of Female Homicide Victims by Years");
plt.xlabel("Year");
plt.ylabel("Numbers of Incident");
plt.legend(relations);

## The End.